In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()

db = client.nyc_restaurants

for rest in db.restaurants.find({"name":"Burger King"}):
    print("BK placed in {} {}".format(rest["borough"], rest["address"]["street"]))


BK placed in Queens Northern Boulevard
BK placed in Queens Northern Boulevard
BK placed in Queens Northern Boulevard
BK placed in Queens Hillside Avenue
BK placed in Brooklyn Church Avenue
BK placed in Queens Queens Boulevard
BK placed in Queens Rockaway Boulevard
BK placed in Queens 61 Avenue
BK placed in Staten Island Jewett Avenue
BK placed in Queens 82 Street
BK placed in Queens Astoria Boulevard
BK placed in Bronx White Plains Road
BK placed in Manhattan Beaver Street
BK placed in Staten Island Willow Brook Road
BK placed in Staten Island Hylan Boulevard
BK placed in Staten Island Hylan Boulevard
BK placed in Staten Island Richmond Avenue
BK placed in Brooklyn South Conduit Avenue
BK placed in Brooklyn Marcy Avenue
BK placed in Manhattan West  125 Street
BK placed in Queens Main Street
BK placed in Queens Merrick Boulevard
BK placed in Brooklyn Hillel Place
BK placed in Bronx East  149 Street
BK placed in Queens Atlantic Avenue
BK placed in Brooklyn 4 Avenue
BK placed in Brooklyn 

In [3]:
from pandas.io.json import json_normalize

bks = pd.DataFrame(db.restaurants.find({"name":"Burger King"}))

display(bks)
display(bks["address"])

locations = json_normalize(bks["address"])
display(locations)


,_id,address,borough,cuisine,grades,name,restaurant_id
0,56b5b0bf7853e8294156f146,"{'building': '22210', 'coord': [-73.759249, 40...",Queens,Hamburgers,"[{'date': 2014-12-18 00:00:00, 'grade': 'A', '...",Burger King,40370167
1,56b5b0bf7853e8294156f148,"{'building': '6815', 'coord': [-73.89707140000...",Queens,Hamburgers,"[{'date': 2014-03-12 00:00:00, 'grade': 'A', '...",Burger King,40370238
2,56b5b0bf7853e8294156f14a,"{'building': '14474', 'coord': [-73.8206678, 4...",Queens,Hamburgers,"[{'date': 2014-02-13 00:00:00, 'grade': 'A', '...",Burger King,40370239
3,56b5b0bf7853e8294156f15c,"{'building': '25810', 'coord': [-73.7007390999...",Queens,Hamburgers,"[{'date': 2014-07-16 00:00:00, 'grade': 'B', '...",Burger King,40370916
4,56b5b0bf7853e8294156f15e,"{'building': '9006', 'coord': [-73.9195106, 40...",Brooklyn,Hamburgers,"[{'date': 2014-12-02 00:00:00, 'grade': 'A', '...",Burger King,40370917
5,56b5b0bf7853e8294156f175,"{'building': '92-83', 'coord': [-73.8664386, 4...",Queens,Hamburgers,"[{'date': 2014-01-29 00:00:00, 'grade': 'A', '...",Burger King,40372422
6,56b5b0bf7853e8294156f188,"{'building': '154-05', 'coord': [-73.780574, 4...",Queens,Hamburgers,"[{'date': 2014-09-08 00:00:00, 'grade': 'A', '...",Burger King,40372618
7,56b5b0bf7853e8294156f461,"{'building': '24202', 'coord': [-73.7378856, 4...",Queens,Hamburgers,"[{'date': 2014-05-20 00:00:00, 'grade': 'A', '...",Burger King,40398389
8,56b5b0bf7853e8294156f605,"{'building': '950', 'coord': [-74.131142, 40.6...",Staten Island,Hamburgers,"[{'date': 2015-01-16 00:00:00, 'grade': 'A', '...",Burger King,40513416
9,56b5b0bf7853e8294156f668,"{'building': '3707', 'coord': [-101.8945214, 3...",Queens,American,"[{'date': 2014-06-04 00:00:00, 'grade': 'A', '...",Burger King,40534067


0     {'building': '22210', 'coord': [-73.759249, 40...
1     {'building': '6815', 'coord': [-73.89707140000...
2     {'building': '14474', 'coord': [-73.8206678, 4...
3     {'building': '25810', 'coord': [-73.7007390999...
4     {'building': '9006', 'coord': [-73.9195106, 40...
5     {'building': '92-83', 'coord': [-73.8664386, 4...
6     {'building': '154-05', 'coord': [-73.780574, 4...
7     {'building': '24202', 'coord': [-73.7378856, 4...
8     {'building': '950', 'coord': [-74.131142, 40.6...
9     {'building': '3707', 'coord': [-101.8945214, 3...
10    {'building': '33-10', 'coord': [-73.9164713999...
11    {'building': '4275', 'coord': [-73.8565856, 40...
12    {'building': '16', 'coord': [-74.0124913, 40.7...
13    {'building': '20', 'coord': [-74.144395, 40.62...
14    {'building': '1565', 'coord': [-74.089108, 40....
15    {'building': '2800', 'coord': [-74.1152092, 40...
16    {'building': '2557', 'coord': [-74.16819079999...
17    {'building': '624', 'coord': [-73.8665421,

,building,coord,street,zipcode
0,22210,"[-73.759249, 40.761574]",Northern Boulevard,11361
1,6815,"[-73.89707140000002, 40.7543896]",Northern Boulevard,11377
2,14474,"[-73.8206678, 40.7647441]",Northern Boulevard,11354
3,25810,"[-73.70073909999999, 40.73895539999999]",Hillside Avenue,11004
4,9006,"[-73.9195106, 40.6527642]",Church Avenue,11236
5,92-83,"[-73.8664386, 40.7319364]",Queens Boulevard,11374
6,154-05,"[-73.780574, 40.6679415]",Rockaway Boulevard,11434
7,24202,"[-73.7378856, 40.7541547]",61 Avenue,11362
8,950,"[-74.131142, 40.613148]",Jewett Avenue,10314
9,3707,"[-101.8945214, 33.5197474]",82 Street,11372


In [91]:
from pandas.io.json import json_normalize

def prepareForTableau(bks):
    locations = json_normalize(bks["address"])
    gelocs = locations.apply(lambda e: [e["coord"][0],e["coord"][1]], result_type="expand", axis=1)
    clean_df = pd.concat([bks["restaurant_id"],locations[["street","zipcode"]],gelocs], axis=1)
    clean_df.rename({1:"lat",0:"long"}, axis=1, inplace=True)
    return clean_df

In [92]:
bks = pd.DataFrame(db.restaurants.find({"name":"Burger King"}))
bks.head()

clean_df = prepareForTableau(bks)
display(clean_df.head())


,restaurant_id,street,zipcode,long,lat
0,40370167,Northern Boulevard,11361,-73.759249,40.761574
1,40370238,Northern Boulevard,11377,-73.897071,40.754390
2,40370239,Northern Boulevard,11354,-73.820668,40.764744
3,40370916,Hillside Avenue,11004,-73.700739,40.738955
4,40370917,Church Avenue,11236,-73.919511,40.652764


In [90]:
clean_df.to_json("bk_rests.json",orient="records")

In [102]:

nearLocation = {
    "lng": 40.717432,
    "lat": -73.994943
}

bks = pd.DataFrame(db.restaurants.find({
    "address.coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 2000, # In meters
     }
   }
}))
bks.head()



OperationFailure: error processing query: ns=restaurants.restaurantsTree: GEONEAR  field=address.coord maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error: unable to find index for $geoNear query

In [101]:
clean_df = prepareForTableau(bks)
display(clean_df.head())
clean_df.to_json("bk_rests_filtered.json",orient="records")

,restaurant_id,street,zipcode,long,lat
0,50011542,Bowery,10002,-73.995023,40.717325
1,41696848,Hester Street,10002,-73.994962,40.717214
2,41137336,Chrystie Street,10002,-73.994584,40.717339
3,41615445,Chrystie Street,10002,-73.994584,40.717339
4,41537020,Chrystie Street,10002,-73.994526,40.717451
